In [37]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/road-traffic-video-monitoring/british_highway_traffic.mp4
/kaggle/input/road-traffic-video-monitoring/dhaka_traffic.mp4
/kaggle/input/road-traffic-video-monitoring/road_trafifc.mp4
/kaggle/input/road-traffic-video-monitoring/traffic_video.avi
/kaggle/input/road-traffic-video-monitoring/traffic_detection.mp4


In [ ]:
!pip install ultralytics

In [ ]:
import os
import shutil
import cv2
import numpy as np
import pandas as pd
from IPython.display import Video
from ultralytics import YOLO
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import animation, rc
rc('animation', html='jshtml')

In [ ]:
!mkdir yolov8_tracking
!mkdir yolov8_tracking/examples
!mkdir yolov8_tracking/examples/weights

In [ ]:
os.chdir('/kaggle/working/yolov8_tracking')
shutil.copy('/kaggle/input/road-traffic-video-monitoring/dhaka_traffic.mp4', '/kaggle/working/yolov8_tracking/sample.mp4')
input_video_path='/kaggle/working/yolov8_tracking/sample.mp4'

In [ ]:
Video(input_video_path, width=640, height=400, embed=True)

In [ ]:
!yolo mode=track tracker="bytetrack.yaml" model=yolov8n.pt conf=0.1 source={input_video_path} save=True save_txt=True

In [ ]:
output_video_path='/kaggle/working/yolov8_tracking/runs/detect/track/sample.avi'
!ffmpeg -y -i {output_video_path} -vf "fps=10" -loop 0 detections.gif

In [ ]:
from IPython.display import Image
Image(open('./detections.gif','rb').read())

In [ ]:
txt_dir='/kaggle/working/yolov8_tracking/runs/detect/track/labels'
txtpaths=[]
texts=os.listdir(txt_dir)
for item in texts:
    txtpaths+=[os.path.join(txt_dir,item)]

In [ ]:
boxdata=[]
boxfile=[]
for i in range(len(txtpaths)):
    file=txtpaths[i]
    boxdata+=[np.loadtxt(file)] 
    filename=file.split('/')[-1]
    if filename=='output.txt':
        filename='output_0.txt'
    boxfile+=[filename[0:-4]]
#print(boxdata[0:1])
#print(boxfile)

In [ ]:
BOX=pd.DataFrame(columns=range(7))

for i in range(len(boxdata)):
    if type(boxdata[i][0])==np.float64:
        add=pd.DataFrame([boxdata[i]])
        add[6]=boxfile[i]
        BOX=pd.concat([BOX,add],axis=0)
    else:
        add=pd.DataFrame(boxdata[i])
        add[6]=boxfile[i]
        #print(add)
        #print(add.shape)        
        BOX=pd.concat([BOX,add],axis=0)       

BOX2=BOX.iloc[:,0:7][BOX[0]==2]
BOX2[0:5]

In [ ]:
BOX2.columns=['class','x','y','w','h','id','file']
#print(BOX2['file'].unique().tolist())
BOX2['order']=BOX2['file'].apply(lambda x:x.split('_')[-1].zfill(3))
#print(BOX2['order'].unique().tolist())
data0=BOX2.sort_values('order',ascending=True).reset_index(drop=True)
display(data0)

In [ ]:
ids=sorted(data0['id'].unique().tolist())
orders=sorted(data0['order'].unique().tolist())
orders=pd.DataFrame(columns=['order'],data=orders)

In [ ]:
!mkdir /kaggle/working/frame
!mkdir /kaggle/working/frame2

In [ ]:
def video_2_frames(video_file=input_video_path, image_dir='/kaggle/working/frame/', image_file='img_%s.png'):
    i = 0
    cap = cv2.VideoCapture(video_file)
    while(cap.isOpened()):
        flag, frame = cap.read()
        if flag == False:
            break
        cv2.imwrite(image_dir+image_file % str(i).zfill(3), frame) 
        i += 1
    cap.release()

In [ ]:
paths=[]
for dirname, _, filenames in os.walk('/kaggle/working/frame'):
    for filename in filenames:
        paths+=[(os.path.join(dirname, filename))]
paths=sorted(paths)
print(paths[0:3])

In [ ]:
data0